## Setup and Import Libraries

In [18]:
import os
from langchain_neo4j import Neo4jGraph, GraphCypherQAChain
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [11]:
load_dotenv()

True

In [12]:
os.environ["NEO4J_URI"] = os.getenv("NEO4J_URI")
os.environ["NEO4J_USERNAME"] = os.getenv("NEO4J_USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("NEO4J_PASSWORD")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [4]:
graph = Neo4jGraph()

In [13]:
llm = ChatGroq(model_name="Gemma2-9b-It")

## Data Insertion

In [7]:
movie_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [8]:
graph.query(query=movie_query)

[]

In [9]:
graph.refresh_schema()
print(graph.schema)

Node properties:
CEO {POB: STRING, name: STRING, YOB: INTEGER}
Company {name: STRING}
datascientist {POB: STRING, name: STRING, YOB: INTEGER}
Country {name: STRING}
Person {name: STRING, born: INTEGER}
Movie {title: STRING, released: INTEGER, id: STRING, imdbRating: FLOAT}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)


## Querying The Graph

In [21]:
chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True
)

In [22]:
query = "Who played in Jumanji?"

response = chain.invoke({"query": query})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Jumanji"})<-[:ACTED_IN]-(p:Person)
RETURN p.name
Full Context:
[{'p.name': 'Robin Williams'}, {'p.name': 'Bradley Pierce'}, {'p.name': 'Kirsten Dunst'}, {'p.name': 'Jonathan Hyde'}]

> Finished chain.


{'query': 'Who played in Jumanji?',
 'result': 'Robin Williams, Bradley Pierce, Kirsten Dunst, and Jonathan Hyde played in Jumanji. \n'}

In [24]:
query = "Who directed the movie Toy Story"

response = chain.invoke({"query": query})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Toy Story"})<-[:DIRECTED]-(p:Person) RETURN p.name
Full Context:
[{'p.name': 'John Lasseter'}]

> Finished chain.


{'query': 'Who directed the movie Toy Story',
 'result': 'John Lasseter directed the movie Toy Story. \n'}

In [26]:
query = "Who were the actors in movie Casino"

response = chain.invoke({"query": query})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Casino"})<-[:ACTED_IN]-(a:Person) RETURN a.name 

Full Context:
[{'a.name': 'Robert De Niro'}, {'a.name': 'Joe Pesci'}, {'a.name': 'Sharon Stone'}, {'a.name': 'James Woods'}]

> Finished chain.


{'query': 'Who were the actors in movie Casino',
 'result': 'Robert De Niro, Joe Pesci, Sharon Stone, and James Woods were in the movie Casino.  \n'}